In [ ]:
import pandas as pd

# 1. Load the CardTransaction data
# Adjust the path to your CSV file
try:
    df_transactions = pd.read_csv('CardTransaction.csv')
    print("CardTransaction.csv loaded successfully.")
    print(f"Initial shape: {df_transactions.shape}")
except FileNotFoundError:
    print("Error: CardTransaction.csv not found. Please check the file path.")
    exit()

# 2. Convert Time Columns to Datetime Objects
# Ensure proper format if your times are not standard (e.g., 'HH:MM:SS')
# The snippet you showed had '38:30:00', which is problematic if it's not a standard time format.
# Assuming standard 'HH:MM:SS' or 'YYYY-MM-DD HH:MM:SS' after your Power Query experience.
# If they are just 'HH:MM:SS', you might need to combine with a dummy date.
# For simplicity, let's assume they are 'YYYY-MM-DD HH:MM:SS' strings from Power Query.
# If they are just 'HH:MM:SS' you'll need to parse them more carefully.

# Let's assume EntranceTime and ExitTime columns are in a format like 'YYYY-MM-DD HH:MM:SS' or similar,
# even if the snippet showed unusual numbers. You cleaned them in Power Query, so they should be usable.
# If they are still just times like '01:31:44', you'd need to combine them with the date first.

# The safest way is to ensure your CSV after Power Query has the correct DateTime format.
# If not, let's load the *original* CSV and apply the Power Query logic here.

# --- Re-implementing Power Query Date Cleaning in Pandas ---
# Make sure your original EntranceTime and ExitTime columns are loaded correctly.
# If they are not strings in the CSV, pandas will try to infer.
# If they are strings, specify format, or let pandas infer:
df_transactions['EntranceTime'] = pd.to_datetime(df_transactions['EntranceTime'], errors='coerce')
df_transactions['ExitTime'] = pd.to_datetime(df_transactions['ExitTime'], errors='coerce')

# Impute missing times based on NoEntry/NoExit flags
# Rule 1: No EntranceTime (NoEntry = 1) -> Midnight of ExitTime's day
# Use .loc to avoid SettingWithCopyWarning
mask_no_entry = df_transactions['NoEntry'] == 1
df_transactions.loc[mask_no_entry, 'EntranceTime'] = df_transactions.loc[mask_no_entry, 'ExitTime'].dt.normalize()

# Rule 2: No ExitTime (NoExit = 1) -> Midnight after EntranceTime's day
mask_no_exit = df_transactions['NoExit'] == 1
df_transactions.loc[mask_no_exit, 'ExitTime'] = df_transactions.loc[mask_no_exit, 'EntranceTime'].dt.normalize() + pd.Timedelta(days=1)

# Handle cases where BOTH are NaN after imputation (if any, though rules cover most)
# This might occur if a row had NaN for both EntranceTime AND ExitTime initially,
# and NoEntry/NoExit flags were not 1 (which shouldn't happen based on rules).
df_transactions.dropna(subset=['EntranceTime', 'ExitTime'], inplace=True)


# Filter data to the specified range (January 1, 2021 - April 30, 2025)
start_date = pd.to_datetime('2021-01-01')
end_date = pd.to_datetime('2025-04-30')
df_transactions = df_transactions[(df_transactions['EntranceTime'] >= start_date) &
                                  (df_transactions['EntranceTime'] <= end_date)] # Filter by EntranceTime


print(f"Shape after date cleaning and filtering: {df_transactions.shape}")
print(df_transactions[['EntranceTime', 'ExitTime', 'NoEntry', 'NoExit']].head())

# --- Next, we'd proceed with the aggregation logic ---
# This is the tricky part.
# We'll create a new DataFrame to store hourly occupancy.

# Find the overall min and max datetimes to define our intervals
overall_min_dt = df_transactions['EntranceTime'].min()
overall_max_dt = df_transactions['ExitTime'].max() # Use max exit time for full range

# Define intervals (e.g., hourly intervals)
# We need intervals that span from the start of the first entry hour to the end of the last exit hour
# Let's create a list of all hours in your data range
hourly_intervals = pd.date_range(start=overall_min_dt.floor('H'),
                                 end=overall_max_dt.ceil('H'),
                                 freq='H')

# Create a DataFrame to store the aggregated occupancy
# This will be the result that we save to CSV
hourly_occupancy_df = []

# Iterate through each LotNumber
for lot_num in df_transactions['LotNumber'].unique():
    # Filter transactions for the current lot
    lot_transactions = df_transactions[df_transactions['LotNumber'] == lot_num]

    # Iterate through each hourly interval
    for current_hour_start in hourly_intervals:
        current_hour_end = current_hour_start + pd.Timedelta(hours=1)

        # Count active cars in this lot during this hour
        # A car is active if its entrance time is <= current_hour_end
        # AND its exit time is >= current_hour_start
        active_cars = lot_transactions[
            (lot_transactions['EntranceTime'] <= current_hour_end) &
            (lot_transactions['ExitTime'] >= current_hour_start)
        ]

        # Count distinct card numbers
        active_count = active_cars['CardNumber'].nunique()

        # Append to our results list
        hourly_occupancy_df.append({
            'DateTimeInterval': current_hour_start,
            'LotNumber': lot_num,
            'ActiveCarsCount': active_count
        })

# Convert the list of dictionaries to a DataFrame
hourly_occupancy_final_df = pd.DataFrame(hourly_occupancy_df)

print(f"Shape of aggregated occupancy data: {hourly_occupancy_final_df.shape}")
print(hourly_occupancy_final_df.head())

# Save the aggregated data to a new CSV file
# This is the file you'd load into Power BI
hourly_occupancy_final_df.to_csv('AggregatedOccupancy.csv', index=False)
print("\nAggregatedOccupancy.csv created successfully.")

CardTransaction.csv loaded successfully.
Initial shape: (4246646, 9)
Shape after date cleaning and filtering: (4242447, 9)
         EntranceTime            ExitTime  NoEntry  NoExit
1 2021-01-01 00:00:00 2021-01-01 08:34:31        1       0
2 2021-01-01 00:00:00 2021-01-01 09:38:30        1       0
3 2021-01-01 12:31:44 2021-01-01 13:16:59        0       0
4 2021-01-01 14:07:08 2021-01-01 15:01:53        0       0
5 2021-01-02 00:00:00 2021-01-02 06:26:17        1       0


<ipython-input-1-f83a704df8c0>:72: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_intervals = pd.date_range(start=overall_min_dt.floor('H'),
<ipython-input-1-f83a704df8c0>:73: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  end=overall_max_dt.ceil('H'),


Shape of aggregated occupancy data: (1100405, 3)
     DateTimeInterval  LotNumber  ActiveCarsCount
0 2021-01-01 00:00:00         21                2
1 2021-01-01 01:00:00         21                2
2 2021-01-01 02:00:00         21                2
3 2021-01-01 03:00:00         21                2
4 2021-01-01 04:00:00         21                2

AggregatedOccupancy.csv created successfully.
